In [1]:
from pyspark.streaming.kafka import KafkaUtils
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import from_json, col
from datetime import datetime

In [2]:
sc.stop()

import os
os.chdir("..")
os.chdir("..")
BASE_PATH = os.getcwd()

TOPIC = "topic_one"
BOOTSTRAP_SERVER = "localhost:9092"
ZOOKEEPER = "localhost:2181"

In [3]:
# os.environ["PYSPARK_SUBMIT_ARGS"] = f"--packages org.apache.spark:spark-sql-kafka-0-10_2.11 org.apache.kafka:kafka-clients-2.4.0 pyspark-shell"

In [3]:
spark = SparkSession\
    .builder\
    .appName("StructuredNetworkWordCount")\
    .master("local")\
    .config("spark.jars", f"file://{BASE_PATH}/jars/spark-sql-kafka-0-10_2.11-2.4.4.jar,file://{BASE_PATH}/jars/kafka-clients-2.4.0.jar") \
    .getOrCreate()

In [6]:
sample_data = spark.read.json(f"file://{BASE_PATH}/00_input/data/subkingdoms.json")
sample_schema = sample_data.schema

In [8]:
sample_schema

StructType(List(StructField(id,LongType,true),StructField(link,StringType,true),StructField(name,StringType,true),StructField(slug,StringType,true)))

In [9]:
 stream_df = spark\
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", BOOTSTRAP_SERVER) \
        .option("subscribe", TOPIC) \
        .load()

In [11]:
# string_df = stream_df.selectExpr("CAST(value AS STRING)")
query_df = stream_df.select( \
  from_json(col("value").cast("string"), sample_schema))

In [12]:
query_df.printSchema()

root
 |-- jsontostructs(CAST(value AS STRING)): struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- link: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- slug: string (nullable = true)



In [13]:
# Start running the query that prints the running counts to the console
query = query_df \
    .writeStream \
    .format("console") \
    .start()

In [ ]:
query.awaitTermination()